In [1]:
# Check if the easydiffraction library is installed.
# If not, install it with the 'visualization' extras.
# Needed when running remotely (e.g. Colab) where the lib is absent.
import builtins
import importlib.util

if (hasattr(builtins, '__IPYTHON__') and
    importlib.util.find_spec('easydiffraction') is None):
    !pip install 'easydiffraction[visualization]'


# Structure Refinement: Si, SEPD

This example demonstrates a Rietveld refinement of Si crystal
structure using time-of-flight neutron powder diffraction data from
SEPD at Argonne.

## Import Library

In [2]:
from easydiffraction import ExperimentFactory
from easydiffraction import Project
from easydiffraction import SampleModelFactory
from easydiffraction import download_data

## Define Sample Model

This section shows how to add sample models and modify their
parameters.

#### Create Sample Model

In [3]:
model = SampleModelFactory.create(name='si')

#### Set Space Group

In [4]:
model.space_group.name_h_m = 'F d -3 m'
model.space_group.it_coordinate_system_code = '2'

#### Set Unit Cell

In [5]:
model.cell.length_a = 5.431

#### Set Atom Sites

In [6]:
model.atom_sites.add(
    label='Si',
    type_symbol='Si',
    fract_x=0.125,
    fract_y=0.125,
    fract_z=0.125,
    b_iso=0.5,
)

## Define Experiment

This section shows how to add experiments, configure their
parameters, and link the sample models defined in the previous step.

#### Download Measured Data

In [7]:
data_path = download_data(id=7, destination='data')

Getting data...


Data #7: Si, SEPD (Argonne)


✅ Data #7 downloaded to 'data/ed-7.xye'


#### Create Experiment

In [8]:
expt = ExperimentFactory.create(name='sepd', data_path=data_path, beam_mode='time-of-flight')

Data loaded successfully


Experiment 🔬 'sepd'. Number of data points: 5600


#### Set Instrument

In [9]:
expt.instrument.setup_twotheta_bank = 144.845
expt.instrument.calib_d_to_tof_offset = 0.0
expt.instrument.calib_d_to_tof_linear = 7476.91
expt.instrument.calib_d_to_tof_quad = -1.54

#### Set Peak Profile

In [10]:
expt.peak_profile_type = 'pseudo-voigt * ikeda-carpenter'
expt.peak.broad_gauss_sigma_0 = 3.0
expt.peak.broad_gauss_sigma_1 = 40.0
expt.peak.broad_gauss_sigma_2 = 2.0
expt.peak.broad_mix_beta_0 = 0.04221
expt.peak.broad_mix_beta_1 = 0.00946

Peak profile type for experiment 'sepd' changed to


pseudo-voigt * ikeda-carpenter


#### Set Peak Asymmetry

In [11]:
expt.peak.asym_alpha_0 = 0.0
expt.peak.asym_alpha_1 = 0.5971

#### Set Background

In [12]:
expt.background_type = 'line-segment'
for x in range(0, 35000, 5000):
    expt.background.add(id=str(x), x=x, y=200)

Background type for experiment 'sepd' changed to


line-segment


#### Set Linked Phases

In [13]:
expt.linked_phases.add(id='si', scale=10.0)

## Define Project

The project object is used to manage the sample model, experiment, and
analysis.

#### Create Project

In [14]:
project = Project()

#### Add Sample Model

In [15]:
project.sample_models.add(sample_model=model)

#### Add Experiment

In [16]:
project.experiments.add(experiment=expt)

## Perform Analysis

This section shows the analysis process, including how to set up
calculation and fitting engines.

#### Set Calculator

In [17]:
project.analysis.current_calculator = 'cryspy'

Current calculator changed to


cryspy


#### Set Minimizer

In [18]:
project.analysis.current_minimizer = 'lmfit (leastsq)'

Current minimizer changed to


lmfit (leastsq)


#### Plot Measured vs Calculated

In [19]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 1/5

Set parameters to be refined.

In [20]:
model.cell.length_a.free = True

expt.linked_phases['si'].scale.free = True
expt.instrument.calib_d_to_tof_offset.free = True

Show free parameters after selection.

In [21]:
project.analysis.show_free_params()

Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,si,cell,,length_a,5.43100,,-inf,inf,Å
2,sepd,linked_phases,si,scale,10.00000,,-inf,inf,
3,sepd,instrument,,d_to_tof_offset,0.00000,,-inf,inf,µs


#### Run Fitting

In [22]:
project.analysis.fit()

Using experiment 🔬 'sepd' for 'single' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


,iteration,χ²,improvement [%]
1,1,113.06,
2,7,72.20,36.1% ↓
3,11,66.76,7.5% ↓
4,30,66.72,


<IPython.core.display.Javascript object>

...

📏 Weighted R-factor (wR): 12.54%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,si,cell,,length_a,5.4310,5.4314,0.0002,Å,0.01 % ↑
2,sepd,linked_phases,si,scale,10.0000,13.3619,0.1153,,33.62 % ↑
3,sepd,instrument,,d_to_tof_offset,0.0000,-9.2543,0.2503,µs,N/A


#### Plot Measured vs Calculated

In [23]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [24]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 2/5

Set more parameters to be refined.

In [25]:
for point in expt.background:
    point.y.free = True

Show free parameters after selection.

In [26]:
project.analysis.show_free_params()

Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,si,cell,,length_a,5.43135,0.00018,-inf,inf,Å
2,sepd,linked_phases,si,scale,13.36187,0.11531,-inf,inf,
3,sepd,instrument,,d_to_tof_offset,-9.25432,0.25033,-inf,inf,µs
4,sepd,background,0,y,200.00000,,-inf,inf,
5,sepd,background,5000,y,200.00000,,-inf,inf,
6,sepd,background,10000,y,200.00000,,-inf,inf,
7,sepd,background,15000,y,200.00000,,-inf,inf,
8,sepd,background,20000,y,200.00000,,-inf,inf,
9,sepd,background,25000,y,200.00000,,-inf,inf,
10,sepd,background,30000,y,200.00000,,-inf,inf,


#### Run Fitting

In [27]:
project.analysis.fit()

Using experiment 🔬 'sepd' for 'single' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


,iteration,χ²,improvement [%]
1,1,66.80,
2,14,3.38,94.9% ↓
3,48,3.38,


<IPython.core.display.Javascript object>

...

📏 Weighted R-factor (wR): 5.99%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,si,cell,,length_a,5.4314,5.4314,0.0000,Å,0.00 % ↑
2,sepd,linked_phases,si,scale,13.3619,14.6317,0.0265,,9.50 % ↑
3,sepd,instrument,,d_to_tof_offset,-9.2543,-9.2534,0.0515,µs,0.01 % ↓
4,sepd,background,0,y,200.0000,268.6002,0.9745,,34.30 % ↑
5,sepd,background,5000,y,200.0000,144.7589,0.4071,,27.62 % ↓
6,sepd,background,10000,y,200.0000,120.0247,0.4282,,39.99 % ↓
7,sepd,background,15000,y,200.0000,135.8494,0.8169,,32.08 % ↓
8,sepd,background,20000,y,200.0000,132.6887,1.4317,,33.66 % ↓
9,sepd,background,25000,y,200.0000,175.1775,2.8755,,12.41 % ↓
10,sepd,background,30000,y,200.0000,180.4556,5.8525,,9.77 % ↓


#### Plot Measured vs Calculated

In [28]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [29]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 3/5

Fix background points.

In [30]:
for point in expt.background:
    point.y.free = False

Set more parameters to be refined.

In [31]:
expt.peak.broad_gauss_sigma_0.free = True
expt.peak.broad_gauss_sigma_1.free = True
expt.peak.broad_gauss_sigma_2.free = True

Show free parameters after selection.

In [32]:
project.analysis.show_free_params()

Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,si,cell,,length_a,5.43137,0.00004,-inf,inf,Å
2,sepd,linked_phases,si,scale,14.63167,0.02651,-inf,inf,
3,sepd,peak,,gauss_sigma_0,3.00000,,-inf,inf,µs²
4,sepd,peak,,gauss_sigma_1,40.00000,,-inf,inf,µs/Å
5,sepd,peak,,gauss_sigma_2,2.00000,,-inf,inf,µs²/Å²
6,sepd,instrument,,d_to_tof_offset,-9.25336,0.05153,-inf,inf,µs


#### Run Fitting

In [33]:
project.analysis.fit()

Using experiment 🔬 'sepd' for 'single' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


,iteration,χ²,improvement [%]
1,1,3.38,
2,10,3.21,5.0% ↓
3,39,3.21,


<IPython.core.display.Javascript object>

...

📏 Weighted R-factor (wR): 4.81%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,si,cell,,length_a,5.4314,5.4314,0.0000,Å,0.00 % ↑
2,sepd,linked_phases,si,scale,14.6317,14.7057,0.0257,,0.51 % ↑
3,sepd,peak,,gauss_sigma_0,3.0000,5.7727,0.4206,µs²,92.42 % ↑
4,sepd,peak,,gauss_sigma_1,40.0000,44.2827,0.7966,µs/Å,10.71 % ↑
5,sepd,peak,,gauss_sigma_2,2.0000,1.2962,0.1680,µs²/Å²,35.19 % ↓
6,sepd,instrument,,d_to_tof_offset,-9.2534,-9.2506,0.0546,µs,0.03 % ↓


#### Plot Measured vs Calculated

In [34]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [35]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)

### Perform Fit 4/5

Set more parameters to be refined.

In [36]:
model.atom_sites['Si'].b_iso.free = True

Show free parameters after selection.

In [37]:
project.analysis.show_free_params()

Free parameters for both sample models (🧩 data blocks) and experiments (🔬 data blocks)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,value,uncertainty,min,max,units
1,si,cell,,length_a,5.43143,0.00004,-inf,inf,Å
2,si,atom_site,Si,b_iso,0.50000,,-inf,inf,Å²
3,sepd,linked_phases,si,scale,14.70568,0.02567,-inf,inf,
4,sepd,peak,,gauss_sigma_0,5.77274,0.42055,-inf,inf,µs²
5,sepd,peak,,gauss_sigma_1,44.28265,0.79664,-inf,inf,µs/Å
6,sepd,peak,,gauss_sigma_2,1.29621,0.16795,-inf,inf,µs²/Å²
7,sepd,instrument,,d_to_tof_offset,-9.25065,0.05458,-inf,inf,µs


#### Run Fitting

In [38]:
project.analysis.fit()

Using experiment 🔬 'sepd' for 'single' fitting


🚀 Starting fit process with 'lmfit (leastsq)'...


📈 Goodness-of-fit (reduced χ²) change:


,iteration,χ²,improvement [%]
1,1,3.21,
2,44,3.19,


<IPython.core.display.Javascript object>

...

📏 Weighted R-factor (wR): 4.34%


📈 Fitted parameters:


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,datablock,category,entry,parameter,start,fitted,uncertainty,units,change
1,si,cell,,length_a,5.4314,5.4314,0.0000,Å,0.00 % ↑
2,si,atom_site,Si,b_iso,0.5000,0.5201,0.0034,Å²,4.02 % ↑
3,sepd,linked_phases,si,scale,14.7057,14.8594,0.0366,,1.05 % ↑
4,sepd,peak,,gauss_sigma_0,5.7727,4.8233,0.4394,µs²,16.45 % ↓
5,sepd,peak,,gauss_sigma_1,44.2827,45.2122,0.8016,µs/Å,2.10 % ↑
6,sepd,peak,,gauss_sigma_2,1.2962,1.1931,0.1667,µs²/Å²,7.95 % ↓
7,sepd,instrument,,d_to_tof_offset,-9.2506,-9.2823,0.0548,µs,0.34 % ↑


#### Plot Measured vs Calculated

In [39]:
project.plot_meas_vs_calc(expt_name='sepd', show_residual=True)

In [40]:
project.plot_meas_vs_calc(expt_name='sepd', x_min=23200, x_max=23700, show_residual=True)